In [ ]:
from pycocotools.coco import COCO
import os
import shutil
import yaml

base_dir = r"D:\test"
annotations_dir = os.path.join(base_dir, "annotations")
output_dit = os.path.join(base_dir, "vehicles_dataset")

COCO_PATHS = {
    "train": {
        "images": os.path.join(base_dir, "train2017"),
        "ann_file": os.path.join(annotations_dir, "instances_train2017.json")
    },
    "val": {
        "images": os.path.join(base_dir, "val2017"),
        "ann_file": os.path.join(annotations_dir, "instances_val2017.json")
    }
}

CATEGORIES = ['bus',"car", "motorcycle",'truck']

def extract_vehicles(coco_mode):
    print(f"\nОбработка {coco_mode} данных...")
    coco = COCO(COCO_PATHS[coco_mode]["ann_file"])
    
    cat_ids = coco.getCatIds(catNms=CATEGORIES)
    print(f"Категории: {CATEGORIES} -> ID: {cat_ids}")
    
    img_ids = set()
    for cat_id in cat_ids:
        img_ids.update(coco.getImgIds(catIds=cat_id))
    
    print(f"Найдено изображений: {len(img_ids)}")
    
    images_dir = os.path.join(output_dit, coco_mode, "images")
    labels_dir = os.path.join(output_dit, coco_mode, "labels")
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)
    
    for img_id in img_ids:
        img_info = coco.loadImgs(img_id)[0]
        src_img = os.path.join(COCO_PATHS[coco_mode]["images"], img_info["file_name"])
        dst_img = os.path.join(images_dir, img_info["file_name"])
        
        if os.path.exists(src_img):
            shutil.copy(src_img, dst_img)
            
            ann_ids = coco.getAnnIds(imgIds=img_id, catIds=cat_ids)
            anns = coco.loadAnns(ann_ids)
            
            txt_file = os.path.splitext(img_info["file_name"])[0] + ".txt"
            dst_txt = os.path.join(labels_dir, txt_file)
            
            with open(dst_txt, "w") as f:
                for ann in anns:
                    x, y, w, h = ann["bbox"]
                    img_w, img_h = img_info["width"], img_info["height"]
                    
                    x_center = (x + w / 2) / img_w
                    y_center = (y + h / 2) / img_h
                    w_norm = w / img_w
                    h_norm = h / img_h

                    f.write(f"0 {x_center} {y_center} {w_norm} {h_norm}\n")

    return images_dir

def create_yaml_config(train_path, val_path):
    """Создает data.yaml для YOLO"""
    yaml_path = os.path.join(output_dit, "data.yaml")
    
    config = {
        "path": output_dit,
        "train": os.path.relpath(train_path, output_dit),
        "val": os.path.relpath(val_path, output_dit),
        "names": {0: "vehicle"},
        "nc": 1
    }
    
    with open(yaml_path, 'w') as f:
        yaml.dump(config, f, sort_keys=False)
    
    print(f"\nСоздан YOLO конфиг: {yaml_path}")
    return yaml_path

def main():
    os.makedirs(output_dit, exist_ok=True)
    
    train_path = extract_vehicles("train")
    val_path = extract_vehicles("val")
    yaml_path = create_yaml_config(train_path, val_path)

    train_labels = os.path.join(output_dit, "train", "labels")
    val_labels = os.path.join(output_dit, "val", "labels")
    
    print(f"  - Train: {len(os.listdir(train_path))} изображений, {len(os.listdir(train_labels))} аннотаций")
    print(f"  - Val: {len(os.listdir(val_path))} изображений, {len(os.listdir(val_labels))} аннотаций")
    print(f"  - YAML: {yaml_path}")

if __name__ == "__main__":
    main()

In [ ]:
from ultralytics import YOLO
import os


DATA_YAML = r"D:\test\vehicles_dataset\data.yaml"
MODEL_TYPE = "yolov8l.pt" 
EPOCHS = 30
IMGSZ = 540
BATCH = 16  
DEVICE = "cuda"  

model = YOLO(MODEL_TYPE)
results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device=DEVICE,
    name="vehicles_detection",
    exist_ok=True
)


New https://pypi.org/project/ultralytics/8.3.161 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.152  Python-3.10.0 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\test\vehicles_dataset\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=540, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=vehicles_detection, nbs=64, nms=Fa

c:\Users\galiy\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\cuda\__init__.py:287: UserWarning: 
NVIDIA GeForce RTX 5070 Ti with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_90 compute_37.
If you want to use the NVIDIA GeForce RTX 5070 Ti GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  3   4723712  ultralytics.nn.modules.block.C2f             [1024, 512, 3]                
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  3   1247744  ultralytics.nn.modules.block.C2f             [768, 256, 3]                 
 16                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
 17            [-1, 12]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

train: Scanning D:\test\vehicles_dataset\train\labels... 18175 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18175/18175 [00:17<00:00, 1026.85it/s]

train: D:\test\vehicles_dataset\train\images\000000099844.jpg: 2 duplicate labels removed


...

      30/30      8.78G      0.894     0.5935      1.076         80        544: 100%|██████████| 1136/1136 [08:30<00:00,  2.23it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.49it/s]

                   all        796       3003      0.826      0.657      0.762      0.538



30 epochs completed in 4.344 hours.
Optimizer stripped from runs\detect\vehicles_detection\weights\last.pt, 87.6MB
Optimizer stripped from runs\detect\vehicles_detection\weights\best.pt, 87.6MB

Validating runs\detect\vehicles_detection\weights\best.pt...
Ultralytics 8.3.152  Python-3.10.0 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
Model summary (fused): 112 layers, 43,607,379 parameters, 0 gradients, 164.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.45it/s]


                   all        796       3003      0.824      0.658      0.763      0.538
Speed: 0.1ms preprocess, 4.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\vehicles_detection


In [ ]:

from ultralytics import YOLO
import os

BEST_MODEL_PATH = os.path.join("runs", "detect", "vehicles_detection", "weights", "best.pt")

model = YOLO(BEST_MODEL_PATH)
DATA_YAML = r"D:\test\vehicles_dataset\data.yaml"

VAL_BATCH = 16
VAL_IMGSZ = 640


metrics = model.val(
    data=DATA_YAML,
    batch=VAL_BATCH,
    imgsz=VAL_IMGSZ,
    device=DEVICE,
    split="val", 
    name="vehicles_validation",
    exist_ok=True
)


print("\n=== Результаты валидации ===")
print(f"mAP@0.5: {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")



Ultralytics 8.3.152  Python-3.10.0 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
Model summary (fused): 112 layers, 43,607,379 parameters, 0 gradients, 164.8 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1909.8452.4 MB/s, size: 153.4 KB)


val: Scanning D:\test\vehicles_dataset\val\labels.cache... 796 images, 0 backgrounds, 0 corrupt: 100%|██████████| 796/796 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:07<00:00,  6.85it/s]


                   all        796       3003      0.822      0.675      0.776      0.549
Speed: 0.1ms preprocess, 6.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\vehicles_validation

=== Результаты валидации ===
mAP@0.5: 0.7763
mAP@0.5:0.95: 0.5490


In [ ]:
import cv2
import time
import numpy as np
from collections import defaultdict, deque
from ultralytics import YOLO

class BarrierFSM:
    def __init__(self):
        self.STATE_IDLE = "IDLE"
        self.STATE_OPENING = "OPENING"
        self.STATE_OPEN = "OPEN"
        self.STATE_CLOSING = "CLOSING"
        self.state = self.STATE_IDLE
        self.target_id = None
        self.downward_time = defaultdict(float)
        self.disappear_time = 0
        self.last_activity_time = time.time()

        # Параметры
        self.OPEN_THRESH = 1    # секунд спуска
        self.CLOSE_DELAY = 5     # сек
        self.INACTIVITY_TIMEOUT = 120  # сек
        self.OPEN_CLOSE_DUR = 1   # сек
        self.MIN_ANGLE = 30       # град

    def update(self, current_time, tracked_objects, dt):
        # обновление активности
        if tracked_objects:
            self.last_activity_time = current_time

        # таймаут бездействия
        if current_time - self.last_activity_time >= self.INACTIVITY_TIMEOUT:
            if self.state not in [self.STATE_IDLE, self.STATE_CLOSING]:
                self.state = self.STATE_CLOSING
                self.trigger_time = current_time
                print(f"[{current_time:.2f}] Принудительное закрытие по таймауту бездействия")

        if self.state == self.STATE_IDLE:
            for obj_id, _, movement_y, angle in tracked_objects:
                if movement_y > 0 and abs(angle) >= self.MIN_ANGLE:
                    self.downward_time[obj_id] += dt
                    if self.downward_time[obj_id] >= self.OPEN_THRESH:
                        self.state = self.STATE_OPENING
                        self.target_id = obj_id
                        self.trigger_time = current_time
                        print(f"[{current_time:.2f}] Открытие по длительному спуску объекта {obj_id}")
                else:
                    self.downward_time[obj_id] = 0

        elif self.state == self.STATE_OPENING:
            if current_time - self.trigger_time >= self.OPEN_CLOSE_DUR:
                self.state = self.STATE_OPEN
                print(f"[{current_time:.2f}] Шлагбаум открыт")

        elif self.state == self.STATE_OPEN:
            target_found = any(obj_id == self.target_id for obj_id, *_ in tracked_objects)
            if not target_found:
                if self.disappear_time == 0:
                    self.disappear_time = current_time
                elif current_time - self.disappear_time >= self.CLOSE_DELAY:
                    self.state = self.STATE_CLOSING
                    self.trigger_time = current_time
                    print(f"[{current_time:.2f}] Открытие завершено, начинаем закрытие")
            else:
                self.disappear_time = 0

        elif self.state == self.STATE_CLOSING:
            if current_time - self.trigger_time >= self.OPEN_CLOSE_DUR:
                self.state = self.STATE_IDLE
                self.target_id = None
                self.downward_time.clear()
                self.disappear_time = 0
                print(f"[{current_time:.2f}] Шлагбаум закрыт")


def calculate_movement_angle(track_history, track_id, current_point):
    if track_id not in track_history or len(track_history[track_id]) < 2:
        return 0
    prev_point = track_history[track_id][-1]
    dx = current_point[0] - prev_point[0]
    dy = current_point[1] - prev_point[1]
    if dx == 0:
        return 90.0 if dy > 0 else -90.0
    angle_rad = np.arctan2(dy, dx)
    angle_deg = np.degrees(angle_rad)
    vertical_angle = 90.0 - abs(angle_deg)
    return vertical_angle if dy > 0 else -vertical_angle


def process_video(video_path):
    model = YOLO("runs/detect/vehicles_detection/weights/best.pt")
    barrier = BarrierFSM()
    cap = cv2.VideoCapture(video_path)
    track_history = defaultdict(lambda: deque(maxlen=30))
    prev_time = time.time()

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        current_time = time.time()
        dt = current_time - prev_time
        prev_time = current_time
        h = frame.shape[0]

        results = model.track(frame, persist=True, classes=[0], conf=0.4, verbose=False)
        tracked_objects = []
        forced_open = False

        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu()
            ids = results[0].boxes.id.int().cpu().tolist()
            for box, tid in zip(boxes, ids):
                x1, y1, x2, y2 = box
                cx, cy = float((x1+x2)/2), float((y1+y2)/2)

                # принудительное открытие по линии 1/3
                if cy > h/3 and barrier.state == barrier.STATE_IDLE:
                    barrier.state = barrier.STATE_OPENING
                    barrier.trigger_time = current_time
                    barrier.target_id = tid
                    forced_open = True
                    print(f"[{current_time:.2f}] Принудительное открытие: объект {tid} ниже 1/3")

                prev_pt = track_history[tid][-1] if track_history[tid] else None
                movement_y = cy - prev_pt[1] if prev_pt is not None else 0
                track_history[tid].append((cx, cy))
                angle = calculate_movement_angle(track_history, tid, (cx, cy))
                tracked_objects.append((tid, cy, movement_y, angle))

                # отрисовка
                col = (0, 255, 0)
                if tid == barrier.target_id:
                    col = (0, 0, 255)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), col, 2)
                for i in range(1, len(track_history[tid])):
                    p1 = track_history[tid][i-1]; p2 = track_history[tid][i]
                    cv2.line(frame, (int(p1[0]), int(p1[1])), (int(p2[0]), int(p2[1])), col, 2)
                cv2.putText(frame, f"ID:{tid} A:{angle:.1f}", (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, col, 1)

        if not forced_open:
            barrier.update(current_time, tracked_objects, dt)

        # отображение
        state = barrier.state
        status = "OPEN" if state in [barrier.STATE_OPENING, barrier.STATE_OPEN] else "CLOSED"
        color = (0,255,0) if status=="OPEN" else (0,0,255)
        cv2.putText(frame, f"State: {state}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.putText(frame, f"Barrier: {status}", (10,70), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.line(frame, (0, h//3), (frame.shape[1], h//3), (255,0,0), 2)

        cv2.imshow("Barrier Control System", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Запуск
process_video(r"D:\test\cvtest.avi")

